# Conditionals

## Input

We will now introduce a special instruction, which adds a fully new dimension to our programming language. This instruction will make our programs interactive and capable of handling a dialogue with the user. This will make our programs far more interesting, but at the same time more complex: users are unpredictable, and we can never determine beforehand what the user input will look like. Programs handling input should take this substantial unpredictability into account in the form of additional validation statements, which translates into added complexity of our programs.

We will define a single instruction, `input`, which "somehow receives" a string from the user. We will not focus on the actual mechanism: whether it is an input dialog box, a line typed in a terminal, or something else, it does not matter to us. What does matter is that we expect to find the data from the next user input somewhere in the state.

We will therefore assume that, from now on, the state "contains" the sequence of all inputs that the user will ever produce for the product. This makes our state an "oracle" of sorts, because it knows in advance what will happen with the user. This "trick" is needed when defining how we manage input on paper, because of the fundamental limitation of paper not being an interactive system. This assumption is also not completely ungrounded: we assume that we already know the inputs, but these inputs will indeed be known at some point in time. So what we really do is define how the program did behave, or would behave, given inputs that **did** happen or **would** happen. These past/speculative inputs will be stored in the state. The new structure of the state will therefore be:

$$\{ \dots, \text{input} := \{ \text{next} := \text{"}\dots\text{"}, \text{rest} := \{ \text{next} := \text{"}\dots\text{"}, \text{rest} := \{\dots \} \} \} \} $$

Where the first $\dots$ are just the regular data we would keep in the state, and inside `input` we find a (potentially infinite) stream of values, each of which is a string (stored in a variable called `next`). A possible state with input will therefore look like:

$$\{ \text{input} := \{ \text{next} := \text{"hello"}, \text{rest} := \{ \text{next} := \text{"world"}, \text{rest} := \{\}  \} \}$$

We can therefore just read the next user input by simply stating `S[input][next]`, and we can discard it and move to the next user input by `S[input := S[input][rest]]`.

For example, with respect to the state example above, we could read the current input by stating:

$$\{ \text{input} := \{ \text{next} := \text{"hello"}, \text{rest} := \{ \text{next} := \text{"world"}, \text{rest} := \{\}  \} \}[\text{input}][\text{next}] \rightarrow \text{"hello"}$$

The subsequent input could be read as:

$$\{ \text{input} := \{ \text{next} := \text{"hello"}, \text{rest} := \{ \text{next} := \text{"world"}, \text{rest} := \{\}  \} \}[\text{input}][\text{rest}][\text{next}] \rightarrow \text{"world"}$$

The semantics of `input` are just:

`eval( v := input(), S)` $\rightarrow$ `<done>, S'`

where `S[v := S[input][next]][input := S[input][rest]]` $\rightarrow$ `S'`


Let us assume an initial state of:

$$\{ \text{input} := \{ \text{next} := \text{"Hello"}, \text{rest} := \{ \text{next} := \text{"World"}, \text{rest} := \{ \} \} \} \}$$

We now show the evaluation of the simple program `x := input(); y := input(); z := x + " " + y + "!"; done`.


| Program | State |
|:-------:|:-----:|
| `x := input(); ...` | $\{ \text{input} := \{ \text{next} := \text{"Hello"}, \text{rest} := \{ \text{next} := \text{"World"}, \text{rest} := \{\}  \} \} \}$ |
| `y := input(); ...` | $\{ x := \text{"Hello"}, \text{input} := \{ \text{next} := \text{"World"}, \text{rest} := \{\}  \} \} \} $ |
| `z := x + " " + y + "!"; ...` | $\{ x := \text{"Hello"}, y := \text{"World"}, \text{input} := \{\} \} $ |
| `z := "Hello" + " " + y + "!"; ...` | $\{ x := \text{"Hello"}, y := \text{"World"}, \text{input} := \{\} \} $ |
| `z := "Hello " + y + "!"; ...` | $\{ x := \text{"Hello"}, y := \text{"World"}, \text{input} := \{\} \} $ |
| `z := "Hello " + "World" + "!"; ...` | $\{ x := \text{"Hello"}, y := \text{"World"}, \text{input} := \{\} \} $ |
| `z := "Hello World" + "!"; ...` | $\{ x := \text{"Hello"}, y := \text{"World"}, \text{input} := \{\} \} $ |
| `z := "Hello World!"; ...` | $\{ x := \text{"Hello"}, y := \text{"World"}, \text{input} := \{\} \} $ |
| `done` | $\{ x := \text{"Hello"}, y := \text{"World"}, z := \text{"Hello World!"}, \text{input} := \{\} \} $ |


Similarly we can define variations in the way we read the input in order to read different kind of values, such as: `v := int(input())` when we want to get an integer value as input, `v := bool(input())` when we want a boolean value, etc...

## Conditionals
An obvious and immediate consequence of input is that it makes our programs less predictable. We have no access to the input as we are writing the program, and so the program must be able to adjust its behaviour consistently with respect to the inputs it will receive.

Some instructions will, at some point during the evaluation of a program, become a better response than others. For example, if we ask the age to the user, we could want to give as a response a different message based on the age (like `Hello child!` or `Hello young man!` or `Hello old man!`). This introduces a requirement: our program must be able to choose a statement rather than another based on some dynamical (i.e. only known at *runtime*, that is the period during which a computer program is executing) condition. To achieve this goal we can introduce the fundamental instruction to perform choices, known as the *conditional* instruction or *if-then-else*.

The syntax of this instruction is `if C then P else Q`, where `C` is an expression, and both `P` and `Q` are statements (known as _branches_). We expect `C` to be a boolean expression, that is an expression which will evaluate to either `True` or `False`. Should `C` evaluate to something else, we will interrupt the evaluation as the program gets stuck in an error state. 

When `C` evaluates to `True`, we will then move onto the evaluation of `P`, thereby skipping `Q`. Conversely, when `C` evaluates to `False`, we will then move onto the evaluation of `Q`, thereby skipping `P`. The semantics of `if-then-else` reflect this process and are not particularly intricated:
- `eval(<if True then P else Q>, S)` $\rightarrow$ `(<P>, S)`
- `eval(<if False then P else Q>, S)` $\rightarrow$ `(<Q>, S)`
- `eval(<if C then P else Q>, S)` $\nrightarrow$ where `C` is any non-boolean constant value
- `eval(<if C then P else Q>, S)` $\rightarrow$ `(<if C' then P else Q>, S)` where `eval_expr(<C>, S)` $\rightarrow$  `<C'>`

We will use curly brackets `{` and `}` to denote the boundaries of `p` and `q` (that is, where they begin and end). This yields code looking like:

```
x := int(input()); if x > 0 then { z := "the input was positive"; done } else { z := "the input was not positive"; done }; done
```

Note that we need to use the `int` operator to convert the input from a string to an integer value.

Let us show now the evaluation of the following simple program, assuming that the initial state is $\{ \text{input} := \{ \text{next} := \text{"5"}, \text{rest} := \{\} \} \}$:

```
x := int(input()); if (x + 1) > 5 then { x := x - 1; done } else { x := x + 1; done }; done
```

| Program                  | State           |
|:------------------------:|:---------------:|
| `x := int(input()); if (x + 1) > 5 then { x := x - 1; done } else { x := x + 1; done }; done` | $\{ \text{input} := \{ \text{next} := \text{"5"}, \text{rest} := \{\} \} \}$ |
| `x := int("5"); if (x + 1) > 5 then { x := x - 1; done } else { x := x + 1; done }; done` | $\{ \text{input} := \{\} \}$ |
| `x := 5; if (x + 1) > 5 then { x := x - 1; done } else { x := x + 1; done }; done` | $\{ \text{input} := \{\} \}$ |
| `if (x + 1) > 5 then { x := x - 1; done } else { x := x + 1; done }; done` | $\{ \text{input} := \{\}, x := 5 \}$ |
| `if (5 + 1) > 5 then { x := x - 1; done } else { x := x + 1; done }; done` | $\{ \text{input} := \{\}, x := 5 \}$ |
| `if 6 > 5 then { x := x - 1; done } else { x := x + 1; done }; done` | $\{ \text{input} := \{\}, x := 5 \}$ |
| `if True then { x := x - 1; done } else { x := x + 1; done }; done` | $\{ \text{input} := \{\}, x := 5 \}$ |
| `x := x - 1; done; done` | $\{ \text{input} := \{\}, x := 5 \}$ |
| `x := 5 - 1; done; done` | $\{ \text{input} := \{\}, x := 5 \}$ |
| `x := 4; done; done` | $\{ \text{input} := \{\}, x := 5 \}$ |
| `done; done` | $\{ \text{input} := \{\}, x := 4 \}$ |
| `done` | $\{ \text{input} := \{\}, x := 4 \}$ |


When nesting multiple conditional statements inside each other, then it is clear that the curly brackets make it much easier to understand which of the conditionals we are looking at:

```
x := int(input()); if x > 0 then { z := "the input was positive"; done } else { if x < 0 then { z := "the input was negative"; done } else { z := the input was zero; done} }; done
```

Unfortunately, chaining all statements on a single line is not really a scalable strategy, so we can now make a slight extension to our syntax in order to improve readability. The improvement in readability often elicits a strong emotional response in the reader, because it makes it apparent that a program can (on the surface at least) be complicated by an inadequate notation. It is therefore important to notice that the real complexity lies in the semantics, and since we are not changing the semantics, the improvement to our language is only skin-deep.

At the end of each code block (so also nested inside the conditional branches `p` and `q`) we have a `done` statement. We will, from now on, leave it implicit. This leads us to:

```
x := int(input()); if x > 0 then { z := "the input was positive" } else { if x < 0 then { z := "the input was negative" } else { z := the input was zero } }
``` 

The program is shorter, and a bit less "noisy". We now use introduce the use of whitespace (newline characters) in order to add visual structure. Instead of `;`, we will alternatively use new lines. This results in a much improved version of the program:

```
x := int(input())
if x > 0 then { 
z := "the input was positive" 
} else { 
if x < 0 then { 
z := "the input was negative" 
} else { 
z := the input was zero } }
```

The program now feels significantly more readable, but the visual boundary between the instructions inside conditionals and outside is not clear and requires "chasing the curly braces". Instead of using curly braces to delimit branches and code blocks, we will move onto  the use of indentation. Indentation uses a prefix of blank characters (tabs or spaces) in order to visually emphasize nesting. This results in the final version of our code:
        
```
x := int(input())
if x > 0 then
  z := "the input was positive" 
else
  if x < 0 then 
    z := "the input was negative" 
  else
    z := the input was zero
```

Depending on the context, we will use one or the other syntax (one-liner vs formatted) without thinking about it. It is important to remember (and repeat almost as if it were a mantra): *changes in syntax are not relevant, it is the semantics that count*).

Sometimes we need to chain multiple decisions together, that is we need to perform more controls inside the various `else` clauses. We can, of course, do this by writing some nested conditional statements:

```
todo := input()
x := int(input())
y := int(input())
if todo == "add" then
    z := x + y
else
    if todo == "sub" then
        z := x - y
    else
        if todo == "mul" then
            z := x * y
        else
            if todo == "div" then
                z := x / y
            else
                z := 0
```

Since this is a common scenario, many languages nowadays offer a simple way to put an `if` as the only instruction inside an `else`, that is `elif`/`elseif` (the name suggests indeed the fusion of an `else` and its subsequent `if`):

```
todo := input()
x := int(input())
y := int(input())
if todo == "add" then
  z := x + y
elif todo == "sub" then
  z := x - y
elif todo == "mul" then
  z := x * y
elif todo == "div" then
  z := x / y
else
  z := 0
```

The last two examples do not differ in semantics in any way, and moreover `elif` is only syntactic sugar which gets translated directly into `else if`. For this reason `elif` gets no extra semantic definitions.

Sometimes it could be convenient to not execute any sub-program in the `else` branch. With only the rules introduced so far we would have to write `if C then P else { done }`, which would not be incorrect, but is cumbersome. We will therefore add a new syntactic sugar rule: whenever we do not write the `else` clause, for example in a program such as `if C then P`, we are assuming that `else { done }` is added automatically by the language.



### Expressive power of conditional instructions
We might be wondering how substantial the addition of conditional statements to our programming languages is. This can be measured by quantifying how many possible execution paths can be encoded by a single program with respect to all the possible execution paths that the computer is capable of, regardless of the program.

Recall that we defined a computer as a huge _network of potential computations_. Each of these computations represents evaluating a given program. The computer somehow "receives" the program to run, and then runs it. Running the program generates a sequence of $P \times S$ pairs of program and state, which we call a _state trace_. Our programs so far featured only one evaluation path, that is $(P_0,S_0) \rightarrow (P_1,S_1) \rightarrow (P_2,S_2) \rightarrow (P_3,S_3) \rightarrow \dots $.

<img src="images/program_state_transitions_highlighted.png" alt="Program state transitions highlighted" style="width: 800px;"/>

The addition of input and conditional statements introduces a dramatic change: programs are now able to express multiple possible traces, depending on the combination of given inputs:

<img src="images/conditional_traces.png" alt="Traces of a conditional" style="width: 800px;"/>

A program with a single conditional statement introduces therefore a fork in the traces, yielding two possible traces instead of one. Multiple conditional statements proceed along this way. Suppose we had the following program featuring two conditional statements after each other: `... if C then P else Q; if D then R else S`.

The possible resulting programs after the first conditional statement are, of course, two (disregarding the states):

`P; if D then R else S` and `Q; if D then R else S`. Of course, both of these programs will eventually reach the second conditional, which means that the concrete traces will run one of the following possible sequences of instructions:

`P;R`, or `P;S`, or `Q;R`, or `Q;S`.

Suppose we added yet a third statement after the second. Then we would get, for each of the possible four sequences above, two more, resulting in eight possible traces.

In general, adding a conditional statement has the potential to multiply the number of possible traces of the program by two. This means that with very little code, our programs now encode a lot of possible paths:

<img src="images/single_program_multiple_traces.png" alt="Multiple traces of a single program with conditionals" style="width: 800px;"/>

Consider as a case study a simple program such as:

```
x := int(input())
if x % 2 == 0 then
  z := "yes"
else
  z := "no"
if x % 3 == 0 then
  w := "yes"
else
  w := "no"
```

Of course, since we cannot know in advance what value `x` will receive from the input, at the end of the program we will find one of the following possible states for `z` and `w`: $\{ z := \text{"yes"}, w := \text{"yes"} \}$, $\{ z := \text{"yes"}, w := \text{"no"} \}$, $\{ z := \text{"no"}, w := \text{"yes"} \}$, $\{ z := \text{"no"}, w := \text{"no"} \}$.

Other ways to compose conditional statements produce less than a doubling of the possible traces. For example, a program featuring an `elif` such as:

```
x := int(input())
if x % 2 == 0 then
  z := "yes"
elif x % 3 == 0 then
  z := "almost"
else
  z := "no"
```

would only feature three possible traces when we consider the value of `z`: $\{ z := \text{"yes"} \}$, $\{z := \text{"almost"}\}$, $\{ z := \text{"no"} \}$.

In conclusion, we have opened up our programs to encode much more potential than before. The use of `input` statements gives us unpredictability, that is we do not know in advance exactly what values we will receive as input. The use of conditional expressions makes it possible for the program to encode multiple separate possible programs which are "activated" depending on the values derived from the input.


## Conditional expressions
Conditional statements are not the only form of conditionals. In some cases, we might want to perform a check which only influences (part of) an expression. For example, we might want to add `1` to `x`, but only if `x` is a multiple of three, and then multiply `x` by `2`. We could write a moderately verbose version of this program as:

```
...
if x % 3 == 0 then
  x := (x + 1) * 2
else
  x := x * 2
```

Fortunately, most programming languages (C++, Java, C#, F#, Haskell, Scala, Python, ...) offer a way to perform a conditional check inside an expression, instead of only as a statement. This leads our program to the simpler form of a single expression:

`x := 2 * (if x % 3 == 0 then (x + 1) else x)`

This requires new syntactic and semantic rules for conditional expressions. The syntax of expressions is therefore extended with
`if C then E1 else E2`, where all of `C`, `E1`, and `E2` are expressions (notice that `E1` and `E2` would be statements in the case of conditional statements, whereas here we only deal with expressions).

The semantic rules, defined by `eval_expr` and not `eval`, are:
- `eval_expr(<if True then E1 else E2>, S)` $\rightarrow$ `<E1>`
- `eval_expr(<if False then E1 else E2>, S)` $\rightarrow$ `<E2>`
- `eval_expr(<if C then E1 else E2>, S)` $\nrightarrow$ where `C` is any non-boolean constant value
- `eval_expr(<if C then E1 else E2>, S)` $\rightarrow$ `<if C' then E1 else E2>` (when `C` is an expression and `eval_expr(<C>, S)` $\rightarrow$ `<C'>`)

Note that, while we could easily extend our syntax for conditional expressions with `elif`, we always need to have a closing `else` clause. In case of an empty clause, we would have no way to evaluate the conditional expression to its final value, therefore such a meaningless construct is not allowed, even syntactically.

Not all languages use the same syntax for statement and expression conditional, but the most advanced (such as Scala, Haskell, or F#) do. Two common syntactic forms that we can encounter in practice will be:
- `C ? E1 : E2` (usually found in languages using a C-style syntax: C, C#, Java, JavaScript, TypeScript, etc.);
- `E1 if C else E2` (found in Python-style languages).

Let us see an example evaluation featuring conditional expressions: `y := if x > 0 then 2 * x + 1 else 0; done` given an initial state $\{ x := 10 \}$

| Program | State |
|:-------:|:-----:|
| `y := if x > 0 then 2 * x + 1 else 0` | $\{ x := 10 \}$ |
| `y := if 10 > 0 then 2 * x + 1 else 0` | $\{ x := 10 \}$ |
| `y := if True then 2 * x + 1 else 0` | $\{ x := 10 \}$ |
| `y := 2 * x + 1` | $\{ x := 10 \}$ |
| `y := 2 * 10 + 1` | $\{ x := 10 \}$ |
| `y := 20 + 1` | $\{ x := 10 \}$ |
| `y := 21` | $\{ x := 10 \}$ |
| `done` | $\{ x := 10, y := 21 \}$ |
